## Dataset: Labeled Faces in the Wild ##
## Experiment: (experiment_1) Image based gender classification ##

In [1]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchvision import transforms
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import pandas as pd
import os
import numpy as np
# import cv2
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline

/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sattvik/envs/pytorch_env/lib/py

### Parameters ###

In [2]:
epochs = 50
batch_size = 64
learning_rate = 0.001
# momentum = 0.5
num_workers = 2

data_path = '../data/lfw/data/'


# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/experiment_1')

### Dataset class used to instanciate the data loader ###

In [3]:
class LFWDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_path, attributes_df, transform=None):
        self.attributes_df = attributes_df
        self.data_path = data_path
        self.transform = transform

    def __len__(self):
        return len(self.attributes_df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_path, "lfw_home/lfw_funneled", self.attributes_df.iloc[idx]['person'].replace(' ', '_'),"{}_{:04d}.jpg".format(self.attributes_df.iloc[idx]['person'].replace(' ', '_'),self.attributes_df.iloc[idx]['imagenum']))
#         img = torch.from_numpy(cv2.imread(img_path))
        img = Image.open(img_path, mode='r')
        
        label = self.attributes_df.iloc[idx]['Male']>0
        
        if self.transform:
            img = self.transform(img)

        return img, torch.tensor(label, dtype=torch.float)

### Creating datasets ###

In [4]:
# read file
attributes_df = pd.read_csv(os.path.join(data_path,'lfw_attributes.txt'))

# split data into training val and test
all_names = attributes_df.person.unique()
tt_msk = np.random.rand(len(all_names)) < 0.8
temp_train_names = all_names[tt_msk]
tv_msk = np.random.rand(len(temp_train_names)) < 0.8
train_names = temp_train_names[tv_msk]
val_names = temp_train_names[~tv_msk]
test_names = all_names[~tt_msk]
del all_names, tt_msk, temp_train_names, tv_msk

# create train val and test dataframes
train_df = attributes_df.loc[attributes_df['person'].isin(train_names)]
val_df = attributes_df.loc[attributes_df['person'].isin(val_names)]
test_df = attributes_df.loc[attributes_df['person'].isin(test_names)]
del attributes_df, train_names, val_names, test_names

# create datasets
train_dataset = LFWDataset(data_path, train_df, transform=transforms.Compose([
                                                    transforms.RandomResizedCrop(224),
                                                    transforms.RandomHorizontalFlip(),
                                                    transforms.ToTensor()
                                                    ]))
val_dataset = LFWDataset(data_path, val_df, transform=transforms.Compose([
                                                    transforms.RandomResizedCrop(224),
                                                    transforms.RandomHorizontalFlip(),
                                                    transforms.ToTensor()]))
test_dataset = LFWDataset(data_path, test_df, transform=transforms.Compose([
                                                    transforms.RandomResizedCrop(224),
                                                    transforms.RandomHorizontalFlip(),
                                                    transforms.ToTensor()]))
del train_df, val_df, test_df

print(len(train_dataset), len(val_dataset), len(test_dataset))

7912 1988 3243


### Declaring dataloaders ###

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

print(len(train_dataloader), len(val_dataloader), len(test_dataloader))

124 32 51


### Sanity check ###

In [6]:
# for i , (img, label) in enumerate(train_dataloader):
#     for im, lab in zip(img, label):
#         plt.imshow(np.moveaxis(np.asarray(im), 0, -1))
#         plt.show()
#         print(lab)
#     if i>2:
#         break

### Define Model ###

In [7]:
# model = torch.hub.load('pytorch/vision:v0.5.0', 'resnet18', pretrained=True)
# model.fc = nn.Linear(512, 1)
# model.add_module("act", nn.Sigmoid())

In [8]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.rnet = torch.hub.load('pytorch/vision:v0.5.0', 'resnet18', pretrained=True)
        self.rnet.fc = nn.Linear(512, 1)
#         self.rnet.__init__()

    def forward(self, x):
        x = torch.sigmoid(self.rnet(x))
        return x

In [9]:
model = ResNet().cuda()

Using cache found in /home/sattvik/.cache/torch/hub/pytorch_vision_v0.5.0


### All layers except the last are frozen ###

In [10]:
print(model)

def unfreeze_layer4():
    for p in model.rnet.layer4.parameters():
        p.requires_grad = True

def unfreeze_layer3():
    for p in model.rnet.layer3.parameters():
        p.requires_grad = True

def unfreeze_layer2():
    for p in model.rnet.layer2.parameters():
        p.requires_grad = True

for p in model.parameters():
    p.requires_grad = False

for p in model.rnet.fc.parameters():
    p.requires_grad = True

for p in model.parameters():
    print(p.requires_grad)

unfreeze_layer4()

ResNet(
  (rnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running

In [11]:
# model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# criteria = loss = nn.CrossEntropyLoss()

In [12]:
def perform_validation(val_model, dataloader):
    with torch.no_grad():
        epoch_loss = 0
        epoch_accuracy = 0
        for batch_idx, (data, target) in tqdm_notebook(enumerate(dataloader), total=len(dataloader)):
            # move data batch to GPU
            data = data.cuda()
            target = target.cuda()
            # forward pass
            output = val_model(data)
            loss = F.binary_cross_entropy(output, target.unsqueeze(1))
            # compute average loss an accuracy
            output = output.to('cpu')
            target = target.to('cpu')
            current_acc = torch.tensor(((output>0.5)== torch.tensor(target.unsqueeze(1), dtype=torch.bool)).sum(), dtype=torch.float)/torch.tensor(len(target), dtype=torch.float)
            epoch_loss = ((epoch_loss*batch_idx) + loss.item())/(batch_idx+1)
            epoch_accuracy = ((epoch_accuracy*batch_idx) + current_acc.item())/(batch_idx+1)
    print("val loss: {} and val accuracy: {}".format(epoch_loss, epoch_accuracy))
    return epoch_loss, epoch_accuracy

In [13]:
def perform_training(val_model, dataloader):
    epoch_loss = 0
    epoch_accuracy = 0
    for batch_idx, (data, target) in tqdm_notebook(enumerate(dataloader), total=len(dataloader)):
        # move data batch to GPU
        data = data.cuda()
        target = target.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward pass
        output = val_model(data)
        loss = F.binary_cross_entropy(output, target.unsqueeze(1))
        # backward pass
        loss.backward()
        optimizer.step()
        # compute average loss an accuracy
        output = output.to('cpu')
        target = target.to('cpu')
        current_acc = torch.tensor(((output>0.5)== torch.tensor(target.unsqueeze(1), dtype=torch.bool)).sum(), dtype=torch.float)/torch.tensor(len(target), dtype=torch.float)
        epoch_loss = ((epoch_loss*batch_idx) + loss.item())/(batch_idx+1)
        epoch_accuracy = ((epoch_accuracy*batch_idx) + current_acc.item())/(batch_idx+1)
    print("train loss: {} and train accuracy: {}".format(epoch_loss, epoch_accuracy))
    return epoch_loss, epoch_accuracy

### Initiate training ###

In [14]:
train_losses=[]
train_accuracies=[]
val_losses=[]
val_accuracies=[]
for epoch in range(epochs):
    # unfreeze layers
    if 5 == epoch:
        unfreeze_layer3()
    # run train and val epochs
    print("Epoch: {}".format(epoch))
    model.train()
    train_loss, train_acc = perform_training(model, train_dataloader)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    writer.add_scalar('training loss', train_loss, epoch)
    writer.add_scalar('training accuracy', train_acc, epoch)
    model.eval()
    val_loss, val_acc = perform_validation(model, val_dataloader)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    writer.add_scalar('validation loss', val_loss, epoch)
    writer.add_scalar('validation accuracy', val_acc, epoch)
writer.close()

Epoch: 0


/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



train loss: 0.3184262582611653 and train accuracy: 0.8802923387096774


/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app



val loss: 0.3671841328032315 and val accuracy: 0.8623046875
Epoch: 1


...


val loss: 0.3144556352926884 and val accuracy: 0.90576171875
Epoch: 48



train loss: 0.087473496494274 and train accuracy: 0.967741935483871



val loss: 0.367566510438337 and val accuracy: 0.90380859375
Epoch: 49



train loss: 0.08552241365936014 and train accuracy: 0.9677923389019505



val loss: 0.37103588425088674 and val accuracy: 0.904296875
